In [ ]:
!pip install langchain gradio beautifulsoup4 unstructured jq kaleido sentence-transformers llama-cpp-python chromadb==0.4.14

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 8.

In [ ]:
from langchain_community.document_loaders import UnstructuredHTMLLoader, BSHTMLLoader, TextLoader, JSONLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import LlamaCpp
import glob
import re
import json
import jq

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/Capstone/transcripts')
!ls

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Capstone/transcripts'

In [ ]:
loader = DirectoryLoader('.', glob="*.txt", recursive=True, show_progress=True, loader_cls=TextLoader)
doc = loader.load()

# def metadata_func(record: dict, metadata: dict) -> dict:
#   metadata["source"] = record.get("source")
#   return metadata

# loader_json = DirectoryLoader('sources/', glob="./*.json", loader_cls=JSONLoader,
#                               loader_kwargs={'jq_schema':'.', 'content_key':'content', 'metadata_func':metadata_func})
# doc_json = loader_json.load()

100%|██████████| 832/832 [00:03<00:00, 252.28it/s]


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(doc)

# texts_json = text_splitter.split_documents(doc_json)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
persist_directory = "../chromadb/"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#!pip install --upgrade chromadb==0.4.14

In [ ]:
# Build database from documents
vectordb = Chroma.from_documents(documents = texts, collection_name = "transcripts",
                                 embedding = embeddings, persist_directory = persist_directory)

vectordb.persist()
vectordb = None

In [ ]:
# Load database from persistent store
vectordb = Chroma("transcripts", embedding_function=embeddings,
                  persist_directory=persist_directory,
                  collection_metadata={"hnsw:space":"cosine"})

In [ ]:
vectordb.similarity_search_with_score("Why is Postgres a better database choice?")

[(Document(page_content="to Postgres or move from Mongo to MySQL and you will find that lots of mid-tier\npure cloud applications are leaving. Are leaving. They tried it, they built it,\nthey scaled it, and they said, I'm sorry, this just\ndoesn't scale as well as something like Postgres when you don't\ndemand ACID semantics. When you create tables and you use\nthose tables in a BASE-style way, and then you do things like read replicas. So I call your attention to why it is\nthat I picked Postgres for this class. And that is, Postgres is one of\nthe better NoSQL databases, right? And so it,\nI couldn't have said that in 2012. But now in 2020 and beyond, Postgres is\na good choice for NoSQL applications. That doesn't mean it's the choice for\nall NoSQL applications, or document or key-value store. So Postgres is great. You've learned a lot already\nabout JSON and Postgres and I hope that helps as you go\nforward making good choices. Cheers.\nPRODTE9\n310\n00:17:40,998 --> 00:17:48,498\n

In [ ]:
retriever = vectordb.as_retriever(search_type="mmr",
                                  search_kwargs={"k":5, "fetch_k":20},
                                  #search_type="similarity_score_threshold",
                                  #search_kwargs={'score_threshold":0.5}
                                  )

In [ ]:
model_path = "wherever my model is"

llm_open = LlamaCpp(
    model_path = model_path,
    n_ctx = 32*1024, # 4096 for Llama, 32*1024 for Mistral
    n_gpu_layers=50,
    temperature=0.15,
    top_p=1,
    top_k=40,
    repeat_penalty=1.1,
    max_tokens=1024,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    stream=True
)

In [ ]:
chat_history = []

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_keys="result")

qa_chain = RetrievalQA.from_chain_type(llm=llm_open, chain_type="stuff", memory=memory,
                                       retriever=retriever, return_source_documents=True,
                                       verbose=True)

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
  gr.Markdown(
      f"""
      Welcome!
      """
  )
  chatbot = gr.Chatbot()
  msg = gr.Textbox()
  clear = gr.Button("Clear")
  json = gr.JSON(label="Source Documents", show_label=True)

  def user(user_message, chat_history):
    response = qa_chain(user_message)
    chat_history.append((user_message, response["result"]))
    yield gr.update(value=""), chat_history, response["source_documents"]

  msg.submit(user, [msg, chatbot], [msg, chatbot, json], queue=False)
  clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True, server_name="my.server", server_port=1234)


In [ ]:
def process_llm_response(llm_response):
  print(llm_response["result"])
  for source in llm_response["source_documents"]:
    print(source.metadata)

query = "Here is my question"
llm_response = qa_chain(query)
process_llm_response(llm_response)